# HANDWRITTEN DIGIT CLASSIFICATION USING MNIST DATASET 

In [2]:

from numpy.random import seed
seed(888)
import tensorflow as tf
tf.random.set_seed(404)

In [41]:
import os
import numpy as np
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


# Constants

In [36]:
X_TRAIN_PATH='MNIST/digit_xtrain.csv'
X_TEST_PATH='MNIST/digit_xtest.csv'
Y_TRAIN_PATH='MNIST/digit_ytrain.csv'
Y_TEST_PATH='MNIST/digit_ytest.csv'

NR_CLASSES=10
VALIDATION_SIZE=10000

IMAGE_HEIGHT=28
IMAGE_WIDHT=28
CHANNELS=1

TOTAL_INPUTS=IMAGE_HEIGHT * IMAGE_WIDHT * CHANNELS

# Load dataset

In [4]:
%%time
X_train_all=np.loadtxt(X_TRAIN_PATH,delimiter=',',dtype=int)


Wall time: 1min 3s


In [5]:
%%time
y_train_all=np.loadtxt(Y_TRAIN_PATH,delimiter=',',dtype=int)

Wall time: 474 ms


In [6]:
X_train_all.shape  # 784 = 28 * 28 * 1  image is greay scale so channel=1

(60000, 784)

In [7]:
y_train_all.shape

(60000,)

In [10]:
%%time
X_test=np.loadtxt(X_TEST_PATH,delimiter=',',dtype=int)

Wall time: 8.73 s


In [11]:
%%time
y_test=np.loadtxt(Y_TEST_PATH,delimiter=',',dtype=int)

Wall time: 64 ms


In [12]:
X_test.shape

(10000, 784)

In [13]:
X_test[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [14]:
y_train_all[0] #labels

5

In [15]:
y_train_all[:5]

array([5, 0, 4, 1, 9])

# Data pre-processing

In [18]:
values=y_train_all[:5]

In [19]:
np.eye(10)[values]   #np.eye() makes 10/10 matrix with diagonal elements=1 

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [21]:
np.eye(12)[8]  

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

In [26]:
# RE_SCALING image pixels between 0-1
X_train_all, X_test = X_train_all/255.0 , X_test /255.0

In [27]:
X_train_all.shape
X_test.shape


(10000, 784)

In [ ]:
X_train_all[1] #values in between 0-1

In [28]:
#one hot encoding ie 3 = [0,0,0,1,0,0,0,0,...]
y_train_all=np.eye(NR_CLASSES)[y_train_all]
y_test=np.eye(NR_CLASSES)[y_test]

In [29]:
y_test.shape

(10000, 10)

In [30]:
y_train_all.shape

(60000, 10)

# creating validation set

In [32]:
X_val=X_train_all[:VALIDATION_SIZE]
y_val=y_train_all[:VALIDATION_SIZE]

X_train=X_train_all[VALIDATION_SIZE:]
y_train=y_train_all[VALIDATION_SIZE:]

In [33]:
X_val.shape

(10000, 784)

In [34]:
X_train.shape

(50000, 784)

In [35]:
X_test.shape

(10000, 784)

## Setup tensorflow graph

In [44]:
# defining hyperparameters

n_epochs=10
learning_rate=0.0001

n_hidden1=516
n_hidden2=64

In [54]:
X = tf.placeholder(tf.float32, shape=[None,TOTAL_INPUTS]) #NONE because size in batches decided later
Y = tf.placeholder(tf.float32, shape=[None,NR_CLASSES])

## setting initial weights and biases for hidden layers

In [60]:
# FOR FIRST HIDDEN LAYER

initial_w1=tf.truncated_normal(shape=[TOTAL_INPUTS,512],stddev=0.1,seed=42)
w1=tf.Variable(initial_value=initial_w1)  #storing weights in tensorflow variable

In [61]:
initial_b1=tf.constant(value=0.0,shape=[n_hidden1])

b1=tf.Variable(initial_value=initial_b1) #storing biases in tensorflow variable

In [68]:
layer1_in = tf.matmul(X , w1) + b1

In [69]:
layer1_out = tf.nn.relu(layer1_in)

In [70]:
# FOR SECOND HIDDEN LAYER

initial_w2=tf.truncated_normal(shape=[n_hidden1,n_hidden2],stddev=0.1,seed=42)
w2=tf.Variable(initial_value=initial_w2)

initial_b2=tf.constant(value=0.0,shape=[n_hidden2])
b2=tf.Variable(initial_value=initial_b2)

layer2_in = tf.matmul(layer1_out, w2) + b2
layer2_out = tf.nn.relu(layer2_in)

In [72]:
# FOR OUTPUT LAYER

initial_w3=tf.truncated_normal(shape=[n_hidden2,NR_CLASSES],stddev=0.1,seed=42)
w3=tf.Variable(initial_value=initial_w3)

initial_b3=tf.constant(value=0.0,shape=[NR_CLASSES])
b3=tf.Variable(initial_value=initial_b3)

layer3_in = tf.matmul(layer2_out, w3) + b3
output = tf.nn.softmax(layer3_in)